In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

In [2]:
train = pd.read_csv('Data Mining VU data/training_set_VU_DM_2014.csv')
test = pd.read_csv('Data Mining VU data/test_set_VU_DM_2014.csv')

In [3]:
train_clean = pd.read_csv('processed_data/train_clean.csv')
test_clean = pd.read_csv('processed_data/test_clean.csv')

In [4]:
# smaller samples for faster computing
tr_cl_sample = train_clean.sample(frac=0.5, random_state=0)
te_cl_sample = train_clean.drop(tr_cl_sample.index)

In [5]:
tr_cl_sample.head()

,srch_id,date_time,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,...,star_diff_bool,price_diff,price_diff_bool,visitor_hist_starrating_filled,visitor_hist_adr_usd_filled,willingness_to_pay,booked_clicked_combined,prop_desirability,star_diff_filled,price_diff_filled
930452,62094,2013-01-26 22:10:19,18,31,NaN,NaN,32,47040,4,4.5,...,0,NaN,0,3.604085,167.558620,0,0,0.000000,0.395915,81.501380
1701430,114235,2012-11-13 12:42:21,1,13,NaN,NaN,15,115700,5,4.5,...,0,NaN,0,3.600963,223.067754,0,0,0.022989,1.399037,291.762246
1595996,107254,2013-04-25 10:22:32,16,31,NaN,NaN,31,31574,4,0.0,...,0,NaN,0,3.673218,194.181384,0,0,0.000000,0.326782,12.688616
3367696,226202,2013-02-21 19:02:17,5,219,NaN,NaN,219,33423,3,3.5,...,0,NaN,0,3.171522,166.362238,0,0,0.000000,0.171522,31.362238
3880444,260461,2013-05-02 23:11:06,14,100,NaN,NaN,219,24194,3,4.5,...,0,NaN,0,3.285429,184.321422,0,0,0.014634,0.285429,94.271422


In [6]:
tr_cl_sample.isnull().sum()

srch_id                                 0
date_time                               0
site_id                                 0
visitor_location_country_id             0
visitor_hist_starrating           2353405
visitor_hist_adr_usd              2352835
prop_country_id                         0
prop_id                                 0
prop_starrating                         0
prop_review_score                       0
prop_brand_bool                         0
prop_location_score1                    0
prop_location_score2                    0
prop_log_historical_price               0
position                                0
price_usd                               0
promotion_flag                          0
srch_destination_id                     0
srch_length_of_stay                     0
srch_booking_window                     0
srch_adults_count                       0
srch_children_count                     0
srch_room_count                         0
srch_saturday_night_bool          

In [7]:
#col = [c for c in tr_cl_sample.columns if c not in ['booking_bool', 'date_time', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'gross_bookings_usd', 'star_diff', 'price_diff']]
col = [c for c in tr_cl_sample.columns if c not in ['position', 'date_time', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'gross_bookings_usd', 'star_diff', 'price_diff', 'booking_bool', 'click_bool', 'booked_clicked_combined']]

In [8]:
target = pd.DataFrame(tr_cl_sample['booked_clicked_combined'])
data = pd.DataFrame(tr_cl_sample[col])

clf = RandomForestClassifier()
clf.fit(data, target)

/Users/Tamika/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [9]:
data_test = pd.DataFrame(te_cl_sample[col])
target_test = pd.DataFrame(te_cl_sample['booked_clicked_combined'])

predict = clf.predict(data_test)
f1_ = f1_score(target_test, predict, average='micro')
acc = accuracy_score(target_test, predict)

print('accuracy: %f' %acc)
print('f1-score (micro): %f' %f1_)
f1_ = f1_score(target_test, predict, average='macro')
print('f1-score (macro): %f' %f1_)
f1_ = f1_score(target_test, predict, average='weighted')
print('f1-score (weighted): %f' %f1_)
#f1_ = f1_score(target_test, predict, average='samples')
#print('f1-score (samples): %f' %f1_)

accuracy: 0.954534
f1-score (micro): 0.954534
f1-score (macro): 0.339936
f1-score (weighted): 0.934009


In [14]:
dictionary = dict(zip(col, clf.feature_importances_))
pd.Series(dictionary).to_frame()

,0
orig_destination_distance,0.068766
price_diff_bool,0.000931
price_diff_filled,0.068934
price_usd,0.071564
promotion_flag,0.003853
prop_brand_bool,0.007764
prop_country_id,0.018713
prop_desirability,0.087630
prop_id,0.061586
prop_location_score1,0.049356


In [ ]:
col = [c for c in tr_cl_sample.columns if c not in ['position', 'date_time', 'visitor_hist_starrating', 'visitor_hist_adr_usd', 'gross_bookings_usd', 'star_diff', 'price_diff', 'booking_bool', 'click_bool', 'booked_clicked_combined']]

target = pd.DataFrame(tr_cl_sample['booked_clicked_combined'])
data = pd.DataFrame(tr_cl_sample[col])

clf = RandomForestClassifier()
clf.fit(data, target)

data_test = pd.DataFrame(te_cl_sample[col])
target_test = pd.DataFrame(te_cl_sample['booked_clicked_combined'])

predict = clf.predict(data_test)
f1_ = f1_score(target_test, predict)
acc = accuracy_score(target_test, predict)
print('f1-score: %f' %f1_)
print('accuracy: %f' %acc)